In [3]:
# ================= CELL 1 : IMPORTS & SETTINGS =================
import pandas as pd
import zipfile
import time
import os
import re

start_cell = time.time()

# ----- USER SETTINGS -----
zip_path = r"E:\WOU_BITS INTERN\39 BUS SYSTEM\DQ Components\LLL.zip"
output_excel = r"E:\WOU_BITS INTERN\39 BUS SYSTEM\DQ Components\Combined_Output.xlsx"

TIME_MIN = 1.45
TIME_MAX = 1.55
# -------------------------

combined_data = []

end_cell = time.time()
print(f"Cell 1 Time: {end_cell - start_cell:.4f} sec")

Cell 1 Time: 0.0000 sec


In [5]:
# ================= CELL 2 : CHECK ZIP FILE =================
start_cell = time.time()

with zipfile.ZipFile(zip_path, 'r') as z:
    files = z.namelist()

print("FILES INSIDE ZIP:\n")
for f in files:
    print(" -", f)

end_cell = time.time()
print(f"\nCell 2 Time: {end_cell - start_cell:.4f} sec")

FILES INSIDE ZIP:

 - LLL/
 - LLL/Line_10_11/
 - LLL/Line_10_11/Line_10_11_1_i_b1_DQ.xlsx
 - LLL/Line_10_11/Line_10_11_1_i_DQ.xlsx
 - LLL/Line_10_11/Line_10_11_1_j_b1_DQ.xlsx
 - LLL/Line_10_11/Line_10_11_1_j_DQ.xlsx
 - LLL/Line_10_11/Line_10_11_99_i_b1_DQ.xlsx
 - LLL/Line_10_11/Line_10_11_99_i_DQ.xlsx
 - LLL/Line_10_11/Line_10_11_99_j_b1_DQ.xlsx
 - LLL/Line_10_11/Line_10_11_99_j_DQ.xlsx
 - LLL/Line_10_13/
 - LLL/Line_10_13/Line_10_13_1_i_b1_DQ.xlsx
 - LLL/Line_10_13/Line_10_13_1_i_DQ.xlsx
 - LLL/Line_10_13/Line_10_13_1_j_b1_DQ.xlsx
 - LLL/Line_10_13/Line_10_13_1_j_DQ.xlsx
 - LLL/Line_10_13/Line_10_13_99_i_b1_DQ.xlsx
 - LLL/Line_10_13/Line_10_13_99_i_DQ.xlsx
 - LLL/Line_10_13/Line_10_13_99_j_b1_DQ.xlsx
 - LLL/Line_10_13/Line_10_13_99_j_DQ.xlsx
 - LLL/Line_13_14/
 - LLL/Line_13_14/Line_13_14_1_i_b1_DQ.xlsx
 - LLL/Line_13_14/Line_13_14_1_i_DQ.xlsx
 - LLL/Line_13_14/Line_13_14_1_j_b1_DQ.xlsx
 - LLL/Line_13_14/Line_13_14_1_j_b2_DQ.xlsx
 - LLL/Line_13_14/Line_13_14_1_j_DQ.xlsx
 - LLL/Line_13

In [7]:
# ================= CELL 3 : READ & FILTER =================
start_cell = time.time()

with zipfile.ZipFile(zip_path, 'r') as z:

    for file in z.namelist():

        # ---- Only process CSV or Excel ----
        if not (file.endswith(".csv") or file.endswith(".xlsx")):
            print(f"Skipping non-data file: {file}")
            continue

        print(f"\nProcessing: {file}")

        # ----- READ FILE -----
        try:
            if file.endswith(".csv"):
                df = pd.read_csv(z.open(file))
            else:
                df = pd.read_excel(z.open(file))
        except Exception as e:
            print("Read error:", e)
            continue

        print("Initial rows:", len(df))
        print("Columns:", list(df.columns))

        # ----- CHECK TIME COLUMN -----
        time_col = None

        for col in df.columns:
            if re.search("time", col, re.IGNORECASE):
                time_col = col
                break

        if time_col is None:
            print("⚠ No TIME column found → skipping file")
            continue

        print("Using time column:", time_col)

        # ----- CONVERT TO NUMERIC -----
        df[time_col] = pd.to_numeric(df[time_col], errors='coerce')

        # ----- FILTER -----
        df = df[(df[time_col] >= TIME_MIN) & (df[time_col] <= TIME_MAX)]

        print("Rows after filter:", len(df))

        # ----- APPEND IF NOT EMPTY -----
        if not df.empty:
            combined_data.append(df)
        else:
            print("No rows matched time range")

end_cell = time.time()
print(f"\nCell 3 Time: {end_cell - start_cell:.4f} sec")

Skipping non-data file: LLL/
Skipping non-data file: LLL/Line_10_11/

Processing: LLL/Line_10_11/Line_10_11_1_i_b1_DQ.xlsx
Initial rows: 100001
Columns: ['Time in sec', 'Vd', 'Vq', 'Vq_err', 'Vq_rms', 'Unnamed: 5', 'Unnamed: 6']
Using time column: Time in sec
Rows after filter: 2001

Processing: LLL/Line_10_11/Line_10_11_1_i_DQ.xlsx
Initial rows: 100001
Columns: ['Time in sec', 'Vd', 'Vq', 'Vq_err', 'Vq_rms', 'Unnamed: 5', 'Unnamed: 6']
Using time column: Time in sec
Rows after filter: 2001

Processing: LLL/Line_10_11/Line_10_11_1_j_b1_DQ.xlsx
Initial rows: 100001
Columns: ['Time in sec', 'Vd', 'Vq', 'Vq_err', 'Vq_rms', 'Unnamed: 5', 'Unnamed: 6']
Using time column: Time in sec
Rows after filter: 2001

Processing: LLL/Line_10_11/Line_10_11_1_j_DQ.xlsx
Initial rows: 100001
Columns: ['Time in sec', 'Vd', 'Vq', 'Vq_err', 'Vq_rms', 'Unnamed: 5', 'Unnamed: 6']
Using time column: Time in sec
Rows after filter: 2001

Processing: LLL/Line_10_11/Line_10_11_99_i_b1_DQ.xlsx
Initial rows: 100001
C

In [9]:
# ================= CELL 4 : CONCAT =================
start_cell = time.time()

if len(combined_data) == 0:
    raise ValueError("""
    ❌ NO OBJECTS TO CONCATENATE
    Possible reasons:
    - Time range not present
    - Wrong column name
    - All files skipped
    """)

final_df = pd.concat(combined_data, ignore_index=True)

print("Final rows:", len(final_df))

end_cell = time.time()
print(f"Cell 4 Time: {end_cell - start_cell:.4f} sec")

Final rows: 720360
Cell 4 Time: 0.0594 sec


In [11]:
# ================= CELL 5 : SAVE =================
start_cell = time.time()

final_df.to_excel(output_excel, index=False)

print("Saved to:", output_excel)

end_cell = time.time()
print(f"Cell 5 Time: {end_cell - start_cell:.4f} sec")

Saved to: E:\WOU_BITS INTERN\39 BUS SYSTEM\DQ Components\Combined_Output.xlsx
Cell 5 Time: 109.9411 sec


In [13]:
# ================= CELL 6 : SUMMARY =================
print("\n===== SUMMARY =====")
print("Total files used :", len(combined_data))
print("Total rows final :", len(final_df))
print("Time range       :", TIME_MIN, "to", TIME_MAX)


===== SUMMARY =====
Total files used : 360
Total rows final : 720360
Time range       : 1.45 to 1.55


In [17]:
# ================= CELL 1 : CUSTOM PARSER =================
import time, os, re
start = time.time()

def parse_custom(path):

    name = os.path.basename(str(path))

    # Example: Line_1_39_1_i_DQ.xlsx

    # 1) Fault Line
    m = re.search(r'Line_(\d+_\d+)', name)
    fault_line = m.group(1) if m else None

    # 2) Fault Location → number after fault line
    m2 = re.search(r'Line_\d+_\d+_(\d+)', name)
    location = m2.group(1) if m2 else None

    # 3) Terminal/Bus → i or j
    m3 = re.search(r'Line_\d+_\d+_\d+_([ij])', name)
    terminal = m3.group(1) if m3 else None

    return fault_line, location, terminal


end = time.time()
print(f"Cell 1 Time: {end-start:.4f} sec")

Cell 1 Time: 0.0000 sec


In [21]:
# ================= CELL 3 : READ & FILTER =================
start_cell = time.time()

with zipfile.ZipFile(zip_path, 'r') as z:

    for file in z.namelist():

        # ---- Only process CSV or Excel ----
        if not (file.endswith(".csv") or file.endswith(".xlsx")):
            print(f"Skipping non-data file: {file}")
            continue

        print(f"\nProcessing: {file}")

        # ----- READ FILE -----
        try:
            if file.endswith(".csv"):
                df = pd.read_csv(z.open(file))
            else:
                df = pd.read_excel(z.open(file))
        except Exception as e:
            print("Read error:", e)
            continue

        print("Initial rows:", len(df))
        print("Columns:", list(df.columns))

        # ================== ⭐ NEW LINE ADDED ⭐ ==================
        df["Source_File"] = file
        # =========================================================

        # ----- CHECK TIME COLUMN -----
        time_col = None

        for col in df.columns:
            if re.search("time", col, re.IGNORECASE):
                time_col = col
                break

        if time_col is None:
            print("⚠ No TIME column found → skipping file")
            continue

        print("Using time column:", time_col)

        # ----- CONVERT TO NUMERIC -----
        df[time_col] = pd.to_numeric(df[time_col], errors='coerce')

        # ----- FILTER -----
        df = df[(df[time_col] >= TIME_MIN) & (df[time_col] <= TIME_MAX)]

        print("Rows after filter:", len(df))

        # ----- APPEND IF NOT EMPTY -----
        if not df.empty:
            combined_data.append(df)
        else:
            print("No rows matched time range")

end_cell = time.time()
print(f"\nCell 3 Time: {end_cell - start_cell:.4f} sec")

Skipping non-data file: LLL/
Skipping non-data file: LLL/Line_10_11/

Processing: LLL/Line_10_11/Line_10_11_1_i_b1_DQ.xlsx
Initial rows: 100001
Columns: ['Time in sec', 'Vd', 'Vq', 'Vq_err', 'Vq_rms', 'Unnamed: 5', 'Unnamed: 6']
Using time column: Time in sec
Rows after filter: 2001

Processing: LLL/Line_10_11/Line_10_11_1_i_DQ.xlsx
Initial rows: 100001
Columns: ['Time in sec', 'Vd', 'Vq', 'Vq_err', 'Vq_rms', 'Unnamed: 5', 'Unnamed: 6']
Using time column: Time in sec
Rows after filter: 2001

Processing: LLL/Line_10_11/Line_10_11_1_j_b1_DQ.xlsx
Initial rows: 100001
Columns: ['Time in sec', 'Vd', 'Vq', 'Vq_err', 'Vq_rms', 'Unnamed: 5', 'Unnamed: 6']
Using time column: Time in sec
Rows after filter: 2001

Processing: LLL/Line_10_11/Line_10_11_1_j_DQ.xlsx
Initial rows: 100001
Columns: ['Time in sec', 'Vd', 'Vq', 'Vq_err', 'Vq_rms', 'Unnamed: 5', 'Unnamed: 6']
Using time column: Time in sec
Rows after filter: 2001

Processing: LLL/Line_10_11/Line_10_11_99_i_b1_DQ.xlsx
Initial rows: 100001
C

KeyboardInterrupt: 